# Imports

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.parse.corenlp import CoreNLPDependencyParser
from nltk.parse.corenlp import CoreNLPParser
from nltk.corpus import wordnet as wn
from nltk.probability import FreqDist
from collections import Counter
import math
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# XGBOOST
import xgboost as xgb
# from xgboost import XGBClassifier

# SPACY IMPORT
import spacy
nlp = spacy.load("en_core_web_lg")

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from spacy.symbols import nsubj, VERB

nlp = spacy.load("en_core_web_lg")


In [2]:
from xgboost import XGBClassifier

# Read Data and PreProcess

In [3]:
# same readData from STS.py
def readData(fileName):

    s1 = []
    s2 = []
    score = []
    file = open(fileName, encoding="utf8")
    text = file.readline()
    text = file.read()
    
    # loop to extract a set of two sentences
    for sentence in text.split('\n'):

        # creating two separate lists of the sentences
        # '.rstrip('.') only removes the last period in the sentence
        
        s1.insert(len(s1), (sentence.split('\t')[1].lower()).rstrip('.'))
        s2.insert(len(s1), (sentence.split('\t')[2].lower()).rstrip('.'))
        
        # inserting the score as a separate lists
        score.insert(len(s1), (sentence.split('\t')[3]))

    # print(s1)
    return s1, s2, score


def preprocess(fileName):

    s1, s2, scores = readData(fileName)
    s1_toks = []
    s2_toks = []

    # tokenizing and tagging
    s1_tags = []
    s2_tags = []

    for sentence in s1:
        tokens = nltk.word_tokenize(sentence)
        s1_toks.insert(len(s1_toks), tokens)
        s1_tags.insert(
            len(s1_tags), nltk.pos_tag(tokens))

    for sentence in s2:
        tokens = nltk.word_tokenize(sentence)
        s2_toks.insert(len(s2_toks), tokens)
        s2_tags.insert(
            len(s2_tags), nltk.pos_tag(tokens))
    
    # Remove the unnecessary tuple and keep just the tags
    for i, tag_list in enumerate(s1_tags):
        s1_tags[i] = [tup[1] for tup in tag_list]
    for i, tag_list in enumerate(s2_tags):
        s2_tags[i] = [tup[1] for tup in tag_list]

    # lemmatizing
    s1_lemmas = []
    s2_lemmas = []
    s1_ls = []
    s2_ls = []
    lemmatizer = WordNetLemmatizer()
    for sentence in s1_toks:
        sentence_components = []
        for token in sentence:
            lemmas = lemmatizer.lemmatize(token)
            sentence_components.insert(len(sentence_components), lemmas)
        s1_lemmas.insert(
            len(s1_lemmas), sentence_components)
        s1_ls.insert(len(s1_ls), ' '.join(word for word in sentence_components))
    
    print(s1_ls[2])

    for sentence in s2_toks:
        sentence_components = []
        for token in sentence:
            lemmas = lemmatizer.lemmatize(token)
            sentence_components.insert(len(sentence_components), lemmas)
        s2_lemmas.insert(
            len(s2_lemmas), sentence_components)
        s2_ls.insert(len(s2_ls), ' '.join(word for word in sentence_components))


        
    # Zipping it all together into one object for each word
    s1_word_lists = []
    s2_word_lists = []
    
    for tok_list, lem_list, tag_list in zip(s1_toks, s1_lemmas, s1_tags):
        sentence_words = []
        for tok, lem, tag in zip(tok_list, lem_list, tag_list):
            word = {}
            word['tok'] = tok
            word['lem'] = lem
            word['tag'] = tag
            sentence_words.append(word)
        s1_word_lists.append(sentence_words) 
        
    for tok_list, lem_list, tag_list in zip(s2_toks, s2_lemmas, s2_tags):
        sentence_words = []
        for tok, lem, tag in zip(tok_list, lem_list, tag_list):
            word = {}
            word['tok'] = tok
            word['lem'] = lem
            word['tag'] = tag
            sentence_words.append(word)
        s2_word_lists.append(sentence_words)  
    
    
    
    # Create a corpus object to represent our corpus
    corpus = {}
    corpus["s1"] = {}
    corpus["s2"] = {}
    corpus['scores'] = [int(i) for i in scores]
    
    corpus["s1"]["sentences"] = s1
    corpus["s2"]["sentences"] = s2
    
    corpus["s1"]["tokens"] = s1_toks
    corpus["s2"]["tokens"] = s2_toks
    
    corpus["s1"]["lemmas"] = s1_lemmas
    corpus["s2"]["lemmas"] = s2_lemmas
    
    corpus["s1"]["tags"] = s1_tags
    corpus["s2"]["tags"] = s2_tags
    
    corpus["s1"]["words"] = s1_word_lists
    corpus["s2"]["words"] = s2_word_lists

    corpus["s1"]["ls"] = s1_ls
    corpus["s2"]["ls"] = s2_ls
    
    return corpus


In [4]:
train_data = preprocess("./data/train-set.txt")
# train_data = preprocess("D:\\UTD\\NLPProject\\v2\\semantic-textual-similarity\\data\\train-set.txt")

the fine are part of failed republican effort to force or entice the democrat to return


In [5]:
train_data['s2']['words'][0][0:5]

[{'tok': 'but', 'lem': 'but', 'tag': 'CC'},
 {'tok': 'other', 'lem': 'other', 'tag': 'JJ'},
 {'tok': 'sources', 'lem': 'source', 'tag': 'NNS'},
 {'tok': 'close', 'lem': 'close', 'tag': 'RB'},
 {'tok': 'to', 'lem': 'to', 'tag': 'TO'}]

### Stop Words

In [6]:
stop_words = set(stopwords.words('english'))
tokenized_sentence_list = train_data['s1']['tokens']+train_data['s2']['tokens']
words_filtered = []

# print(words)

# looking through I've noticed there are a number of stop-words that can be added to the set
stop_words.add(',')
stop_words.add('``')
stop_words.add("n't")

for tsl in tokenized_sentence_list:
    for w in tsl:
        if w not in stop_words and w not in words_filtered:
            words_filtered.append(w)

# Data Inspection

### Frequency Distribution

In [7]:
def frequency_distribution(corpus):
    s1_toks = corpus['s1']['tokens']
    s2_toks = corpus['s2']['tokens']    
    freq_dist = FreqDist()
    for i in range(len(s1_toks)):
        for token in (s1_toks[i] + s2_toks[i]):
            freq_dist[token.lower()] += 1
    return freq_dist

In [8]:
freq_dist = frequency_distribution(train_data)

print(freq_dist.most_common(40))

[('the', 5169), (',', 3690), ('of', 2497), ('to', 2133), ('and', 1716), ('a', 1615), ('in', 1573), ('is', 891), ('that', 831), ('on', 820), ('for', 756), ('it', 587), ('this', 579), ('we', 531), ('with', 464), ('be', 459), ('by', 443), ('i', 425), ('which', 403), ('have', 384), ('not', 366), ('at', 343), ('as', 334), ('are', 333), ('has', 319), ('said', 316), ('was', 304), ('european', 287), ("'s", 280), ('from', 261), ('``', 252), ("''", 242), ('will', 233), ('.', 229), ('also', 223), ('its', 194), ('but', 193), ('would', 191), ('all', 188), ('percent', 187)]


### Score Distribution

# TODO
this data is super imbalanced. we likely need to balance it in preprocessing. `https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html`

In [9]:
score_list = [0,0,0,0,0,0]
for s in train_data['scores']:
    score_list[int(s)] += 1
for i in range(0, len(score_list)):
    print("% 1d: % 4d % 6.2f per" %(i, score_list[i], 100*score_list[i]/len(train_data['scores']))) 

 0:    0   0.00 per
 1:   45   3.03 per
 2:   95   6.40 per
 3:  310  20.89 per
 4:  616  41.51 per
 5:  418  28.17 per


## Feature Engineering

This section includes all the code/functions to create features.

### Helper Functions

In [10]:
def remove_duplicate_tokens(token_list):
    blank_list = []
    for w in token_list:
        if w not in blank_list:
            blank_list.append(w)
    return blank_list

In [11]:
def remove_stopwords(token_list):
    blank_list = []
    for w in token_list:
        if w not in stop_words:
            blank_list.append(w)
    return blank_list

### Cosine Similarity (TF-IDF)

This is the same as the spacy similarity. This one is probably less accurate though as I don't believe it's trained from the GloVe w2v model.

In [12]:
def calc_cosine_similarity(s1, s2):

    # remove the stopwords, transform into TF-IDF matrix, then
    tfidf_matrix = TfidfVectorizer(
        stop_words="english").fit_transform([s1, s2])
    
    cos_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # print(tfidf_matrix.toarray())

    cos_sim = cos_sim_matrix[0][1]

    return cos_sim

In [13]:
calc_cosine_similarity('I like some apples', 'I like the pears')

0.33609692727625756

### Spacy Similarity

In [14]:
def calc_spacy_sim(s1, s2):
    s2 = nlp(s2)
    s1 = nlp(s1)
    return s1.similarity(s2)


In [15]:
calc_spacy_sim('I like some apples', 'I like the pears')

0.9361166303666173

### Smooth Inverse Frequency (SIF)

In [16]:
def calc_sif_similarity(s1, s2, a = .001):
    vectorizer = CountVectorizer(stop_words="english")
    X = vectorizer.fit_transform([s1, s2])
    X_arr = X.toarray()
    sif_matrix = []
    for i in range(0, len(X_arr)):
        sif_arr = []
        for j in range(0, len(X_arr[i])):
            word = vectorizer.get_feature_names()[j]
            w = a / (a + freq_dist[word])
            v = X_arr[i][j]
            sif_arr.append(v*w)
        sif_matrix.append(sif_arr)
    sif_cos_sim_matrix = cosine_similarity(sif_matrix, sif_matrix)
    sif_cos_sim = sif_cos_sim_matrix[0][1]
    return sif_cos_sim

In [17]:
calc_sif_similarity('I like some apples', 'I like the pears')

1.4515545128534995e-10

### Simple Overlap

Unique words that are in both sentences divided by the total number of words in both sentences. Does not include stop words.

In [18]:
def calc_basic_overlap(s1_tokens, s2_tokens):
    s1_tokens = remove_stopwords(s1_tokens)
    s1_tokens = remove_duplicate_tokens(s1_tokens)

    s2_tokens = remove_stopwords(s2_tokens)
    s2_tokens = remove_duplicate_tokens(s2_tokens)
    
    overlap = 0
    encountered_words = []
    for word in (s1_tokens+s2_tokens):
        try:
            if word in encountered_words: # we know we have found an overlap
                overlap += 1
            encountered_words.append(word)
        except ValueError:
            # print(word + ' not found in lexicon. Skipping...')
            continue

    avg_sentence_len = len(s1_tokens+s2_tokens) / 2
    
    overlap_normlalized = overlap / avg_sentence_len
    return overlap, overlap_normlalized

### Named Entity Overlap

In [19]:
def named_entity_overlap(s1, s2):    


    # print(s1)
    
    sentence_nlp1= nlp(s1)
    ner1= [(word.text, word.ent_type_) for word in sentence_nlp1 if word.ent_type_]
    
    sentence_nlp2=nlp(s2)
    ner2 = [(word.text, word.ent_type_) for word in sentence_nlp2 if word.ent_type_]
    
    #list_ner = [list(a) for a in zip(ner1, ner2)]
    
    #print(s1[0:5])
    #print(s2[0:5])
    
    #print(ner1)
    #print()
    #print(ner2)
    #print()
    #print(list_ner[0:5])
    overlap = []

    
    da = {k:v for k,v in ner1}
    db = {k:v for k,v in ner2}
    total_length = len(set(ner1+ner2))
    temp = []
    for a in da.keys():
        for b in db.keys():
            if a==b:
                temp.insert(len(temp), a)
    if total_length != 0:
        overlap =len(temp)/total_length
    else:
        overlap = 0.0
    #overlap.insert(len(overlap),[k  for k in da.keys() and db.keys() if da[k] == db[k]] else continue)
    #print()
   # print(overlap[0:5]) #total common named entities / total unique named entities in both sentences

    return overlap

### Head Verb Overlap

In [20]:
def head_verb_overlap(s1, s2):
    s1_nlp = nlp(s1)
    s2_nlp = nlp(s2)

# Finding a verb with a subject from below — good
    verbs1 = []
    for possible_subject in s1_nlp:
        if possible_subject.dep == nsubj and possible_subject.head.pos == VERB and possible_subject.head.text not in verbs1:
            verbs1.append(possible_subject.head.text)
    verbs2 = []
    for possible_subject in s2_nlp:
        if possible_subject.dep == nsubj and possible_subject.head.pos == VERB and possible_subject.head.text not in verbs2:
            verbs2.append(possible_subject.head.text)
    
    overlap = [value for value in verbs1 if value in verbs2]
    verb_overlap = len(overlap)
    
    total_verbs = len(list(set(verbs1+verbs2)))
    if total_verbs == 0:
        return 0 
    else:
        return verb_overlap/total_verbs

### Synset Overlap

# TODO

We may be able to incorporate POS and dependency parsing here as right now i'm just taking the first synset.

In [21]:
def calc_synset_overlap(s1_tokens, s2_tokens):
    s1_tokens = remove_stopwords(s1_tokens)
    s1_tokens = remove_duplicate_tokens(s1_tokens)

    s2_tokens = remove_stopwords(s2_tokens)
    s2_tokens = remove_duplicate_tokens(s2_tokens)
    
#     print(s2_tokens)
#     print(s1_tokens)

    s1_spread = []
    s2_spread = []
    
    for word in s1_tokens:
        for synset in wn.synsets(word):
            for i in range(0, len(synset.lemmas())):
                syn_word = synset.lemmas()[i].name()
                if syn_word not in s1_spread:
                    s1_spread.append(syn_word)

    for word in s2_tokens:
        for synset in wn.synsets(word):
            for i in range(0, len(synset.lemmas())):
                syn_word = synset.lemmas()[i].name()
                if syn_word not in s2_spread:
                    s2_spread.append(syn_word)         
    
    return calc_basic_overlap(s1_spread, s2_spread)
    
calc_synset_overlap(train_data['s1']['tokens'][0], train_data['s2']['tokens'][0])

(144, 0.72544080604534)

In [22]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from nltk.wsd import lesk


from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic

 
def postag_to_synsettag(tag):
    #print(tag)
    if tag.startswith('N'):
        return 'n'
    if tag.startswith('V') or tag == "MD":
        return 'v'
    if tag.startswith('J'):
        return 'a'
    if tag.startswith('R'):
        return 'r'
    return

def tag_to_synset(sent, word, tag):
    wn_tag = postag_to_synsettag(tag)
    if wn_tag is None:
        #print(1)
        return None
    else:
        try:
            #print(2)
            return lesk(sent, word, wn_tag)
        except:
            #print(3)
            return None

def get_synsets(sentence1, sentence2):
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tag_to_synset(sentence1, *tagged_word) for tagged_word in sentence1]
    synsets2 = [tag_to_synset(sentence2, *tagged_word) for tagged_word in sentence2]
    #print(synsets1)
    #print(synsets2)
    # Filter out the Nones
    synsets1 = [syn for syn in synsets1 if syn]
    synsets2 = [syn for syn in synsets2 if syn]
    #print(synsets1)
    #print(synsets2)
    return synsets1, synsets2

def max_similarity_values():
    t1 = "hello"
    t2 = "hello"
    s1, s2 = get_synsets(t1,t2)
    semcor_ic = wordnet_ic.ic('ic-semcor.dat')

    jcn_maxVal = s1[0].jcn_similarity(s2[0], semcor_ic)
    path_maxVal = s1[0].path_similarity(s2[0])
    lin_maxVal = s1[0].lin_similarity(s2[0], semcor_ic)
    res_maxVal = s1[0].res_similarity(s2[0], semcor_ic)
    wup_maxVal = s1[0].wup_similarity(s2[0])
    lch_maxVal = s1[0].lch_similarity(s2[0])
    
    return path_maxVal, wup_maxVal, lin_maxVal, lch_maxVal, res_maxVal, jcn_maxVal


def sentence_path_similarity(sentence1, sentence2):
    synsets1, synsets2 = get_synsets(sentence1, sentence2)
    score, count = 0.0, 0
    best = 0.0
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        for synset2 in synsets2:
            if synset.path_similarity(synset2) is not None and synset.path_similarity(synset2) > best:
                try:
                    best = synset.path_similarity(synset2)
                except TypeError:
                    continue
        # Check that the similarity could have been computed
        if best is not None:
            score += best
            count += 1
    # Average the values
    try:
        score /= count
    except:
        return 0.0
    return score

def sentence_wup_similarity(sentence1, sentence2):
    synsets1, synsets2 = get_synsets(sentence1, sentence2)
    score, count = 0.0, 0
    best = 0.0
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        for synset2 in synsets2:
            if synset.wup_similarity(synset2) is not None and synset.wup_similarity(synset2) > best:
                try:
                    best = synset.wup_similarity(synset2)
                except TypeError:
                    continue
        # Check that the similarity could have been computed
        if best is not None:
            score += best
            count += 1
    # Average the values
    try:
        score /= count
    except:
        return 0.0
    return score
def sentence_lin_similarity(sentence1, sentence2):
    synsets1, synsets2 = get_synsets(sentence1, sentence2)
    score, count = 0.0, 0
    best = 0.0
    semcor_ic = wordnet_ic.ic('ic-semcor.dat')
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        for synset2 in synsets2:
            try:
                if synset.lin_similarity(synset2, semcor_ic) is not None and synset.lin_similarity(synset2, semcor_ic) > best:
                    try:
                        best = synset.lin_similarity(synset2, semcor_ic)
                    except TypeError:
                        continue
            except:
                continue
        # Check that the similarity could have been computed
        if best is not None:
            score += best
            count += 1
    # Average the values
    try:
        score /= count
    except:
        return 0.0
    return score

def sentence_res_similarity(sentence1, sentence2):
    synsets1, synsets2 = get_synsets(sentence1, sentence2)
    score, count = 0.0, 0
    best = 0.0
    semcor_ic = wordnet_ic.ic('ic-semcor.dat')
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        for synset2 in synsets2:
            try:
                if synset.res_similarity(synset2, semcor_ic) is not None and synset.res_similarity(synset2, semcor_ic) > best:
                    try:
                        best = synset.res_similarity(synset2, semcor_ic)
                    except TypeError:
                        continue
            except:
                continue
        # Check that the similarity could have been computed
        if best is not None:
            score += best
            count += 1
    # Average the values
    try:
        score /= count
    except:
        return 0.0
    return score
def sentence_lch_similarity(sentence1, sentence2):
    synsets1, synsets2 = get_synsets(sentence1, sentence2)
    score, count = 0.0, 0
    best = 0.0
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        for synset2 in synsets2:
            try:
                if synset.lch_similarity(synset2) is not None and synset.lch_similarity(synset2) > best:
                    try:
                        best = synset.lch_similarity(synset2)
                    except TypeError:
                        continue
            except:
                continue
        # Check that the similarity could have been computed
        if best is not None:
            score += best
            count += 1
    # Average the values
    try:
        score /= count
    except:
        return 0.0
    return score

def sentence_jcn_similarity(sentence1, sentence2):
    synsets1, synsets2 = get_synsets(sentence1, sentence2)
    score, count = 0.0, 0
    best = 0.0
    semcor_ic = wordnet_ic.ic('ic-semcor.dat')
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        for synset2 in synsets2:
            try:
                if synset.jcn_similarity(synset2, semcor_ic) is not None and synset.jcn_similarity(synset2, semcor_ic) > best:
                    try:
                        best = synset.jcn_similarity(synset2, semcor_ic)
                    except TypeError:
                        continue
            except:
                continue
        # Check that the similarity could have been computed
        if best is not None:
            score += best
            count += 1
    # Average the values
    try:
          score /= count
    except:
          return 0.0
    return score

def symmetric_sentence_path_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_path_similarity(sentence1, sentence2) + sentence_path_similarity(sentence2, sentence1)) / 2 
def symmetric_sentence_wup_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_wup_similarity(sentence1, sentence2) + sentence_wup_similarity(sentence2, sentence1)) / 2 
def symmetric_sentence_res_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_res_similarity(sentence1, sentence2) + sentence_res_similarity(sentence2, sentence1)) / 2 

def symmetric_sentence_lch_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_lch_similarity(sentence1, sentence2) + sentence_lch_similarity(sentence2, sentence1)) / 2 
def symmetric_sentence_lin_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_lin_similarity(sentence1, sentence2) + sentence_lin_similarity(sentence2, sentence1)) / 2 

def symmetric_sentence_jcn_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_jcn_similarity(sentence1, sentence2) + sentence_jcn_similarity(sentence2, sentence1)) / 2 
focus_sentence = "Dolphins are swimming mammals."
sentence = "Dolphins can swim."
print("Path Similarity: ")
print(focus_sentence, sentence, symmetric_sentence_path_similarity(focus_sentence, sentence))
print("LCH Similarity: ")
print(focus_sentence, sentence, symmetric_sentence_lch_similarity(focus_sentence, sentence))
print("LIN Similarity: ")
print(focus_sentence, sentence, symmetric_sentence_lin_similarity(focus_sentence, sentence))
print("JCN Similarity: ")
print(focus_sentence, sentence, symmetric_sentence_jcn_similarity(focus_sentence, sentence))
print("RES Similarity: ")
print(focus_sentence, sentence, symmetric_sentence_res_similarity(focus_sentence, sentence))
print("WUP Similarity: ")
print(focus_sentence, sentence, symmetric_sentence_wup_similarity(focus_sentence, sentence))
print() 

Path Similarity: 
Dolphins are swimming mammals. Dolphins can swim. 1.0
LCH Similarity: 
Dolphins are swimming mammals. Dolphins can swim. 3.6375861597263857
LIN Similarity: 
Dolphins are swimming mammals. Dolphins can swim. 1.0
JCN Similarity: 
Dolphins are swimming mammals. Dolphins can swim. 1e+300
RES Similarity: 
Dolphins are swimming mammals. Dolphins can swim. 1e+300
WUP Similarity: 
Dolphins are swimming mammals. Dolphins can swim. 1.0



# Pipeline

In this section we run the data through the pipeline to get it into the form necessary to create our models.

In [74]:
def pipeline(corpus):
    # TODO add a check to ensure the lengths of these arrays are the same
    # or add the basic processing to pipeline
    
    s1_array = corpus['s1']['sentences']
    s2_array = corpus['s2']['sentences']
    s1_tokens = corpus['s1']['tokens']
    s2_tokens = corpus['s2']['tokens']
    s1_lemmas = corpus['s1']['lemmas']
    s2_lemmas = corpus['s2']['lemmas']
    s1_ls = corpus['s2']['ls']
    s2_ls = corpus['s2']['ls']
    #path_maxVal, wup_maxVal, lin_maxVal, lch_maxVal, res_maxVal, jcn_maxVal = max_similarity_values()

    #print(s1_lemmas[0])
    
    data = []
    for i in range(0, len(s1_array)):
        cos_sim = calc_cosine_similarity(s1_array[i], s2_array[i])
        sif_sim = calc_sif_similarity(s1_array[i], s2_array[i])
        w_overlap, w_norm_overlap = calc_basic_overlap(s1_tokens[i], s2_tokens[i])
        l_overlap, l_norm_overlap = calc_basic_overlap(s1_lemmas[i], s2_lemmas[i])
        spacy_sim = calc_spacy_sim(s1_array[i], s2_array[i])
        syn_overlap, normalized_syn_overlap = calc_synset_overlap(s1_tokens[i], s2_tokens[i])
        path_similarity = symmetric_sentence_path_similarity(s1_ls[i], s2_ls[i]) 
        #wup_similarity = symmetric_sentence_wup_similarity(s1_array[i], s2_array[i]) / wup_maxVal
        #jcn_similarity = symmetric_sentence_jcn_similarity(s1_array[i], s2_array[i]) / jcn_maxVal
        #lch_similarity = symmetric_sentence_lch_similarity(s1_array[i], s2_array[i]) / lch_maxVal
        #lin_similarity = symmetric_sentence_lin_similarity(s1_array[i], s2_array[i]) / lin_maxVal
        #res_similarity = float(symmetric_sentence_res_similarity(s1_array[i], s2_array[i]) / res_maxVal)
        ne_overlap = named_entity_overlap(s1_array[i], s2_array[i])
        verb_overlap = head_verb_overlap(s1_array[i], s2_array[i])
        #data.append([w_norm_overlap, l_norm_overlap, sif_sim, cos_sim, spacy_sim, syn_overlap, normalized_syn_overlap])
        #data.insert(len(data),[w_norm_overlap, l_norm_overlap, spacy_sim, sif_sim, cos_sim, syn_overlap, normalized_syn_overlap, path_similarity, ne_overlap, verb_overlap, len(s1_tokens[i]), len(s2_tokens[i])])
        #data.append([w_norm_overlap, l_norm_overlap, spacy_sim, sif_sim, cos_sim, syn_overlap, normalized_syn_overlap,path_similarity,wup_similarity,lin_similarity,jcn_similarity,lch_similarity,len(s1_tokens[i]), len(s2_tokens[i])])
        data.insert(len(data),[len(s1_tokens), len(s2_tokens), w_norm_overlap, l_norm_overlap, spacy_sim, sif_sim, cos_sim, syn_overlap, normalized_syn_overlap, path_similarity, ne_overlap, len(s1_tokens[i]), len(s2_tokens[i])])

    return data

In [75]:
train_input = pipeline(train_data)
print(train_input[0:5])

[[1484, 1484, 0.6428571428571429, 0.6666666666666666, 0.9699586985720816, 0.4043188683415115, 0.5949218057093537, 144, 0.72544080604534, 1.0, 0.0, 28, 23], [1484, 1484, 0.631578947368421, 0.6, 0.9316565540040632, 0.37040524322972224, 0.474330706497194, 72, 0.6605504587155964, 1.0, 1.0, 10, 16], [1484, 1484, 0.5, 0.5, 0.9247478261787359, 0.1358693286767868, 0.392181175971253, 96, 0.6295081967213115, 1.0, 0.5, 16, 19], [1484, 1484, 0.7333333333333333, 0.7333333333333333, 0.9677497361187998, 0.6935512636502701, 0.668348418668298, 98, 0.7127272727272728, 1.0, 0.3333333333333333, 20, 29], [1484, 1484, 0.24, 0.24, 0.8618764553778161, 4.979960298599938e-10, 0.12170566815950139, 58, 0.5714285714285714, 1.0, 0.0, 18, 18]]


In [76]:
train_data['scores'][0:5]

[4, 4, 3, 3, 2]

In [77]:
train_input

[[1484,
  1484,
  0.6428571428571429,
  0.6666666666666666,
  0.9699586985720816,
  0.4043188683415115,
  0.5949218057093537,
  144,
  0.72544080604534,
  1.0,
  0.0,
  28,
  23],
 [1484,
  1484,
  0.631578947368421,
  0.6,
  0.9316565540040632,
  0.37040524322972224,
  0.474330706497194,
  72,
  0.6605504587155964,
  1.0,
  1.0,
  10,
  16],
 [1484,
  1484,
  0.5,
  0.5,
  0.9247478261787359,
  0.1358693286767868,
  0.392181175971253,
  96,
  0.6295081967213115,
  1.0,
  0.5,
  16,
  19],
 [1484,
  1484,
  0.7333333333333333,
  0.7333333333333333,
  0.9677497361187998,
  0.6935512636502701,
  0.668348418668298,
  98,
  0.7127272727272728,
  1.0,
  0.3333333333333333,
  20,
  29],
 [1484,
  1484,
  0.24,
  0.24,
  0.8618764553778161,
  4.979960298599938e-10,
  0.12170566815950139,
  58,
  0.5714285714285714,
  1.0,
  0.0,
  18,
  18],
 [1484,
  1484,
  0.3333333333333333,
  0.3333333333333333,
  0.8330880954478816,
  3.975117598699085e-09,
  0.14438355527738672,
  25,
  0.2906976744186

## Models

In this section we fit our feature set to a model.

### Decision Tree

In [78]:
dt_classifier = DecisionTreeClassifier(random_state=14, max_depth=8)
dt_classifier.fit(train_input, train_data['scores'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=14,
            splitter='best')

In [79]:
#print(f'Nodes: {dt_classifier.tree_.node_count}')
#print(f'Max Depth: {dt_classifier.tree_.max_depth}')

training_scores = train_data['scores']
print(dt_classifier.score(train_input, training_scores))

0.7142857142857143


### Random Forest

In [80]:
rf_classifier = RandomForestClassifier(random_state=14, n_estimators=100)
rf_classifier.fit(train_input, train_data['scores'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=14, verbose=0, warm_start=False)

### XGBoost

In [81]:
xgboost_model = XGBClassifier(n_jobs=3, 
                              n_estimators=2000, 
                              #max_depth=6,
                              #gamma=.05, 
                              random_state=42)
xgboost_model.fit(np.asarray(train_input), np.asarray(train_data['scores']))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=2000, n_jobs=3, nthread=None,
       objective='multi:softprob', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

# Testing

In [ ]:
dev_data = preprocess("./data/dev-set.txt")
# dev_data = preprocess("D:\\UTD\\NLPProject\\v2\\semantic-textual-similarity\\data\\dev-set.txt")
dev_input = pipeline(dev_data)

let me remind you that our ally include fervent supporter of this tax


In [ ]:
dt_dev_predictions = dt_classifier.predict(dev_input)
rf_dev_predictions = rf_classifier.predict(dev_input)
xgb_dev_predictions = xgboost_model.predict(np.asarray(dev_input))

In [ ]:
# make sure our lengths match up
print(len(dev_data['scores']))
print(len(dt_dev_predictions))
print(len(rf_dev_predictions))
print(len(xgb_dev_predictions))

In [ ]:
from sklearn.metrics import f1_score

def get_metrics(name, predictions, scores):
    correct = 0
    total_error = 0
    for i in range(0, len(predictions)): 
        if predictions[i] == scores[i]:
            correct += 1
        total_error += abs(int(scores[i]) - int(predictions[i]))
    acc = correct / len(predictions) 
    avg_err = total_error / len(predictions)
    
    f1 = f1_score(scores, predictions, average='weighted')
    return name, acc, avg_err, f1


# Ensure all our arrays are full of ints for metrics.
dev_data['scores'] = [int(i) for i in dev_data['scores']] 
rf_dev_predictions = [int(i) for i in rf_dev_predictions] 
xgb_dev_predictions = [int(i) for i in xgb_dev_predictions] 
dt_dev_predictions = [int(i) for i in dt_dev_predictions] 



xgb_metrics = get_metrics('XGBoost', xgb_dev_predictions, dev_data['scores'])
dt_metrics  = get_metrics('Decision Tree', dt_dev_predictions, dev_data['scores'])
rf_metrics = get_metrics('Random Forest', rf_dev_predictions, dev_data['scores'])

df = pd.DataFrame(
    [dt_metrics, rf_metrics, xgb_metrics], 
    columns = ['Model', 'Accuracy', 'Avg Error', 'F-Score']) 
df